## Importando os dados

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive/')
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
|INFO|google.colab serverextension initialized.
|INFO|Serving notebooks from local directory: /content
|INFO|0 active kernels
|INFO|The Jupyter Notebook is running at:
|INFO|http://localhost:8888/
|INFO|Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
1KooEpO7F22cnf2NsW8R5JxAlkgGV64-M#scrollTo=1UVmIIaL1uzd
1KooEpO7F22cnf2NsW8R5JxAlkgGV64-M#scrollTo=1UVmIIaL1uzd
|CRITICAL|received signal 2, stopping
|INFO|Shutting down 0 kernels


In [0]:
## Listando os arquivos
!ls "/content/drive/My Drive/Projetos/IA/raw_data/"
## Copiando os arquivos csv
!cp "/content/drive/My Drive/Projetos/IA/raw_data/guarani.csv" "guarani.csv"
!cp "/content/drive/My Drive/Projetos/IA/raw_data/pt_nvi.csv" "pt_nvi.csv"
!cp "/content/drive/My Drive/Projetos/IA/raw_data/NTLH.csv" "NTLH.csv"

guarani.bbl.mybible.sqlite  guarani.gsheet    NTLH.csv	  pt_nvi.json
guarani.csv		    NTLH.bbl.mybible  pt_nvi.csv


In [0]:
import pandas as pd 
import numpy as np 


guarani = pd.read_csv('guarani.csv',encoding ='utf8')
ntlh = pd.read_csv('NTLH.csv',encoding ='utf8')

raw_guarani_txt = guarani['Scripture']

raw_ntlh_txt = ntlh['Scripture']



### Preprocessando nosso texto

In [0]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


In [0]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

def preprocess(raw_text):
  
 
    
  ## Tensorflow preprocessing
  raw_text = unicode_to_ascii(raw_text.lower().strip())
  
  
  raw_text = re.sub("<.*?>","",raw_text)
  
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  
  raw_text = re.sub(r"([?.!,¿])", r" \1 ", raw_text)
  raw_text = re.sub(r'[" "]+', " ", raw_text)
  
  raw_text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", raw_text)
  raw_text = raw_text.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  text = '<start> ' + raw_text + ' <end>'
  return text
  
 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
guarani = []
pt_br = []
for text_g, text_ntlh in zip(raw_guarani_txt, raw_ntlh_txt):
  guarani.append(preprocess(text_g).encode('utf-8'))
  pt_br.append(preprocess(text_ntlh).encode('utf-8')) 

  



In [0]:
print(pt_br[0])
print(guarani[0])

<start> no comeco deus criou os ceus e a terra . <end>
<start> genesis jypy i kuaxia moises ombopara va ekueyva , yvy ha e opa mba e ikuai va e omoingoaguejypy i ma nhanderuete ojapo raka e yva ha e yvy . <end>


In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

### Usando a classificação com o tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time